In [2]:
from rdflib import RDF, RDFS, OWL, Namespace, Graph, Literal
import brickschema
from brickschema.namespaces import A, BRICK, UNIT


"""

A Brick model is a digital representation of the resources and relationships
inside a building described using the Brick schema.

A Brick model is maintained/query through the abstraction of a Graph. A Graph
consists of triples (subject, predicate, object). Subjects and objects can be
"classes" or "instances of classes" (sometimes called individuals, instances or
entities). Predicates are the "directed edges" in the graph and are sometimes
referred to as such

    +---------+        Predicate         +--------+
    | Subject |------------------------->| Object |
    +---------+                          +--------+

"""

In [3]:

g = brickschema.Graph()


Now that we have the graph object we have to decide what to put into it. At the
very least, we probably want to put in triples describing the "things" in our
building. In Brick, "things" have a name and a class, which tells us what kind
of thing they are.

The Brick schema defines a set of classes that are ready for you to use. Brick
classes includes HVAC equipment, lighting equipment, electrical equipment,
rooms and other spatial elements, zones, and points like sensors and setpoints.
You can find a list of the classes supported by Brick online:
https://brickschema.org/ontology

Before we start adding things to our Brick model, we need to define a namespace.
A "namespace" is a way of grouping information so that the names we choose for
things don't conflict with names in other Brick models (more on that later).

Namespaces are URLs; they do not have to actually point to a real web resource,
but it is of course helpful if they point to some documentation (try going to
https://brickschema.org/schema/Brick#Air_Handler_Unit as an example).

We will choose an arbitrary URL for our namespace and refer to it by the
nickname "bldg" for convenience. "bldg" is also called a "prefix".

In [4]:
bldg = Namespace("http://abacwsbuilding.cardiff.ac.uk/abacws#")
bacnet = Namespace("http://data.ashrae.org/bacnet/2020#")
brick = Namespace("https://brickschema.org/schema/Brick#")
bsh = Namespace("https://brickschema.org/schema/BrickShape#")
dcterms = Namespace("http://purl.org/dc/terms/")
owl = Namespace("http://www.w3.org/2002/07/owl#")
qudt = Namespace("http://qudt.org/schema/qudt/")
qudtqk = Namespace("http://qudt.org/vocab/quantitykind/")
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
ref = Namespace("https://brickschema.org/schema/Brick/ref#")
s223 = Namespace("http://data.ashrae.org/standard223#")
sdo = Namespace("http://schema.org/")
sh = Namespace("http://www.w3.org/ns/shacl#")
skos = Namespace("http://www.w3.org/2004/02/skos/core#")
sosa = Namespace("http://www.w3.org/ns/sosa/")
tag = Namespace("https://brickschema.org/schema/BrickTag#")
unit = Namespace("http://qudt.org/vocab/unit/")
vcard = Namespace("http://www.w3.org/2006/vcard/ns#")
xsd = Namespace("http://www.w3.org/2001/XMLSchema#")


g.bind("bldg", bldg)
g.bind("bacnet", bacnet)
g.bind("brick", brick)
g.bind("bsh", bsh)
g.bind("dcterms", dcterms)
g.bind("owl", owl )
g.bind("qudt", qudt)
g.bind("qudtqk", qudtqk)
g.bind("rdf", rdf)
g.bind("rdfs", rdfs)
g.bind("ref", ref)
g.bind("s223", s223)
g.bind("sdo", sdo)
g.bind("sh", sh)
g.bind("skos", skos)
g.bind("sosa", sosa)
g.bind("tag", tag)
g.bind("unit", unit)
g.bind("vcard", vcard)
g.bind("xsd", xsd)

Here, we tell our graph what the Brick namespace is. This does *not* mean that
the contents of the Brick schema (which is a graph itself) get pulled in. It
simply allows us to refer to classes and relationships that are defined in the
Brick schema.

In [11]:
# BRICK = Namespace("https://brickschema.org/schema/Brick#")
# g.bind("brick", BRICK)
# RDF = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
# g.bind("rdf", RDF)

Now we can add instances to our Brick model (building graph). We use the RDF
namespace (imported above) in addition to the BRICK and BLDG namespaces.
"RDF.type" is the "type" predicate defined within the RDF graph. The RDF.type
predicate is how we create "instances" of Brick classes

In [5]:
rooms_and_floors = {
    "Floor0": ["Room0.01", "Room0.04", "Room0.05", "Room0.06", "Room0.07", "Room0.08", "Room0.10", "Room0.17", "Room0.18",
               "Room0.19", "Room0.20", "Room0.29", "Room0.31", "Room0.32", "Room0.33", "Room0.34"],
    "Floor1": ["Room1.04", "Room1.06", "Room1.07", "Room1.08", "Room1.09", "Room1.25", "Room1.26", "Room1.34", "Room1.37", 
               "Room1.38", "Room1.39", "Room1.40", "Room1.41"],
    "Floor2": ["Room2.01", "Room2.02", "Room2.03","Room2.04", "Room2.05", "Room2.06","Room2.07", "Room2.08", "Room2.09",
               "Room2.10", "Room2.11", "Room2.12", "Room2.13","Room2.14", "Room2.15","Room2.17", "Room2.18", "Room2.19",
               "Room2.20", "Room2.21", "Room2.22","Room2.24", "Room2.26", "Room2.34","Room2.35", "Room2.36", "Room2.47",
               "Room2.48", "Room2.49", "Room2.50","Room2.52", "Room2.53", "Room2.54","Room2.55", "Room2.56", "Room2.57",
               "Room2.58", "Room2.59", "Room2.60","Room2.61", "Room2.62","Room2.44", "Room2.63", "Room2.64", "Room2.65",
               "Room2.66", "Room2.67"],
    "Floor3": ["Room3.01", "Room3.02", "Room3.03","Room3.04", "Room3.05", "Room3.06","Room3.07", "Room3.08", "Room3.09","Room3.10",
               "Room3.11", "Room3.12", "Room3.13","Room3.14", "Room3.15", "Room3.16","Room3.17", "Room3.18", "Room3.20","Room3.22",
               "Room3.26", "Room3.27", "Room3.36","Room3.37", "Room3.38", "Room3.46","Room3.48", "Room3.50", "Room3.51","Room3.52",
               "Room3.53", "Room3.54", "Room3.56","Room3.57", "Room3.58", "Room3.59","Room3.60", "Room3.61", "Room3.62","Room3.63",
               "Room3.64", "Room3.65", "Room3.66","Room3.67", "Room3.68", "Room3.69","Room3.70", "Room3.71"],
    "Floor4": ["Room4.01", "Room4.02", "Room4.03","Room4.05", "Room4.06", "Room4.07","Room4.08", "Room4.09", "Room4.10","Room4.11",
               "Room4.12", "Room4.13", "Room4.14","Room4.15", "Room4.16", "Room4.18","Room4.19", "Room4.20", "Room4.21","Room4.22",
               "Room4.23", "Room4.24", "Room4.34","Room4.35", "Room4.36", "Room4.38","Room4.44", "Room4.47", "Room4.49","Room4.50",
               "Room4.51", "Room4.52", "Room4.53","Room4.55", "Room4.56", "Room4.57","Room4.58", "Room4.59", "Room4.60","Room4.61",
               "Room4.62","Room4.63", "Room4.64", "Room4.65", "Room4.66","Room4.67", "Room4.68", "Room4.69","Room4.70", "Room4.71"],
    "Floor5": ["Room5.01", "Room5.02", "Room5.03","Room5.04", "Room5.05", "Room5.06","Room5.07", "Room5.08", "Room5.09", "Room5.10", "Room5.11", "Room5.12",
               "Room5.13", "Room5.14", "Room5.15","Room5.16", "Room5.17", "Room5.18", "Room5.20","Room5.21", "Room5.22", "Room5.23","Room5.26",
               "Room5.34", "Room5.35", "Room5.36","Room5.44", "Room5.45", "Room5.48","Room5.49", "Room5.50", "Room5.51","Room5.52",
               "Room5.53", "Room5.54", "Room5.56","Room5.57", "Room5.58", "Room5.59","Room5.60", "Room5.61", "Room5.62","Room5.63", "Room5.64", "Room5.65",
               "Room5.66", "Room5.67", "Room5.68","Room5.69", "Room5.70", "Room5.71"]
}


In [6]:
for floor, room_list in rooms_and_floors.items():
    # Use the strings in the datastructure to refer to entities in the Brick model.
    # By putting "BLDG[floor]" into the graph, we implicitly create the entity.
    g.add((bldg[floor], A, BRICK.Floor))
    for room in room_list:
        g.add((bldg[room], A, BRICK.Room))
        g.add((bldg[room], BRICK.isPartOf, bldg[floor]))
        g.add((bldg[floor], BRICK.hasPart, bldg[room]))

In [8]:
# g.serialize("abacws-building-v1.ttl", format="ttl")

<Graph identifier=Nfcb575fd1e1d4ab7908e85f98b515377 (<class 'brickschema.graph.Graph'>)>

In [ ]:
# # (subject, predicate, object)
# g.add((BLDG.AHU1A, RDF.type, BRICK.Air_Handler_Unit))
# # you can use "quotes" to name entities as well
# g.add((BLDG["VAV2-3"], RDF.type, BRICK.Variable_Air_Volume_Box))

We can also add relationships between entities in our Brick model. The
BRICK.feeds relationship indicates a sequence between two pieces of equipment

In [ ]:
# g.add((BLDG.AHU1A, BRICK.feeds, BLDG["VAV2-3"]))

Let's add a few more entities so the graph is more interesting. We will
implement the Brick model for the *blue* entities in the sample graph at
brickschema.org

In [7]:
zone_and_rooms = {
     "north-Zone": ["Room5.01", "Room5.02", "Room5.03","Room5.04", "Room5.05", "Room5.06","Room5.07",  "Room5.09", "Room5.10", "Room5.12",
               "Room5.13", "Room5.14", "Room5.15","Room5.18","Room5.26"],
      "west-Zone": ["Room5.16", "Room5.17", "Room5.18", "Room5.08","Room5.11", "Room5.20","Room5.21", "Room5.22", "Room5.23","Room5.26",
               "Room5.34", "Room5.35","Room5.05"],
      "south-Zone": ["Room5.36","Room5.44", "Room5.45", "Room5.48","Room5.49", "Room5.50", "Room5.51","Room5.52",
               "Room5.53", "Room5.54", "Room5.56","Room5.57"],
      "east-Zone": ["Room5.58", "Room5.59","Room5.60", "Room5.61", "Room5.62","Room5.63", "Room5.64", "Room5.65",
               "Room5.66", "Room5.67", "Room5.68","Room5.69", "Room5.70", "Room5.71","Room5.44"],
      "north-west-Zone": ["Room5.07", "Room5.08", "Room5.09", "Room5.10", "Room5.11", "Room5.12", 
               "Room5.13", "Room5.14", "Room5.15","Room5.16", "Room5.17", "Room5.18", "Room5.20","Room5.21", "Room5.22", "Room5.23","Room5.26"],
      "north-east-Zone": ["Room5.01", "Room5.02","Room5.67", "Room5.68","Room5.69", "Room5.70"],
      "south-west-Zone": ["Room5.34", "Room5.35", "Room5.36"],
      "south-east-Zone": ["Room5.50", "Room5.51","Room5.52", "Room5.53", "Room5.54", "Room5.56","Room5.57", "Room5.58", "Room5.59","Room5.60", "Room5.61", "Room5.62","Room5.63", "Room5.64", "Room5.65",
               "Room5.66"]
}


for zone, room_list in zone_and_rooms.items():
    # Use the strings in the datastructure to refer to entities in the Brick model.
    # By putting "BLDG[floor]" into the graph, we implicitly create the entity.
    g.add((bldg[zone], A, BRICK.zone))
    for room in room_list:
        g.add((bldg[room], A, BRICK.Room))
        g.add((bldg[room], RDFS.label, bldg[room]))
        g.add((bldg[room], BRICK.hasPart, bldg[zone]))


We can "serialize" this model to a file if we want to load it into another program.


In [27]:
# g.serialize("abacws-building-v2.ttl", format="ttl")

<Graph identifier=N98607a952dfc44b1bc4b934ffed89b09 (<class 'brickschema.graph.Graph'>)>

In [9]:
# with open("abacws-building-v2.ttl", "w") as f:
#     # the Turtle format strikes a balance beteween being compact and easy to read
#     f.write(g.serialize(format="ttl"))

Adding sensors

In [8]:
tempsen = ["temp5.01", "temp5.02", "temp5.03","temp5.04", "temp5.05", "temp5.06","temp5.07", "temp5.08", "temp5.09", "temp5.10", "temp5.11", "temp5.12",
               "temp5.13", "temp5.14", "temp5.15","temp5.16", "temp5.17", "temp5.18", "temp5.20","temp5.21", "temp5.22", "temp5.23","temp5.26",
               "temp5.34", "temp5.35", "temp5.36","temp5.44", "temp5.45", "temp5.48","temp5.49", "temp5.50", "temp5.51","temp5.52",
               "temp5.53", "temp5.54", "temp5.56","temp5.57", "temp5.58", "temp5.59","temp5.60", "temp5.61", "temp5.62","temp5.63", "temp5.64", "temp5.65",
               "temp5.66", "temp5.67", "temp5.68","temp5.69", "temp5.70", "temp5.71"]
humsen = ["hum5.01", "hum5.02", "hum5.03","hum5.04", "hum5.05", "hum5.06","hum5.07", "hum5.08", "hum5.09", "hum5.10", "hum5.11", "hum5.12",
               "hum5.13", "hum5.14", "hum5.15","hum5.16", "hum5.17", "hum5.18", "hum5.20","hum5.21", "hum5.22", "hum5.23","hum5.26",
               "hum5.34", "hum5.35", "hum5.36","hum5.44", "hum5.45", "hum5.48","hum5.49", "hum5.50", "hum5.51","hum5.52",
               "hum5.53", "hum5.54", "hum5.56","hum5.57", "hum5.58", "hum5.59","hum5.60", "hum5.61", "hum5.62","hum5.63", "hum5.64", "hum5.65",
               "hum5.66", "hum5.67", "hum5.68","hum5.69", "hum5.70", "hum5.71"]
soundsen = ["sound5.01", "sound5.02", "sound5.03","sound5.04", "sound5.05", "sound5.06","sound5.07", "sound5.08", "sound5.09", "sound5.10", "sound5.11", "sound5.12",
               "sound5.13", "sound5.14", "sound5.15","sound5.16", "sound5.17", "sound5.18", "sound5.20","sound5.21", "sound5.22", "sound5.23","sound5.26",
               "sound5.34", "sound5.35", "sound5.36","sound5.44", "sound5.45", "sound5.48","sound5.49", "sound5.50", "sound5.51","sound5.52",
               "sound5.53", "sound5.54", "sound5.56","sound5.57", "sound5.58", "sound5.59","sound5.60", "sound5.61", "sound5.62","sound5.63", "sound5.64", "sound5.65",
               "sound5.66", "sound5.67", "sound5.68","sound5.69", "sound5.70", "sound5.71"]
hchosen =   ["hcho5.01", "hcho5.02", "hcho5.03","hcho5.04", "hcho5.05", "hcho5.06","hcho5.07", "hcho5.08", "hcho5.09", "hcho5.10", "hcho5.11", "hcho5.12",
               "hcho5.13", "hcho5.14", "hcho5.15","hcho5.16", "hcho5.17", "hcho5.18", "hcho5.20","hcho5.21", "hcho5.22", "hcho5.23","hcho5.26",
               "hcho5.34", "hcho5.35", "hcho5.36","hcho5.44", "hcho5.45", "hcho5.48","hcho5.49", "hcho5.50", "hcho5.51","hcho5.52",
               "hcho5.53", "hcho5.54", "hcho5.56","hcho5.57", "hcho5.58", "hcho5.59","hcho5.60", "hcho5.61", "hcho5.62","hcho5.63", "hcho5.64", "hcho5.65",
               "hcho5.66", "hcho5.67", "hcho5.68","hcho5.69", "hcho5.70", "hcho5.71"]
airqsen= ["airq5.01", "airq5.02", "airq5.03","airq5.04", "airq5.05", "airq5.06","airq5.07", "airq5.08", "airq5.09", "airq5.10", "airq5.11", "airq5.12",
               "airq5.13", "airq5.14", "airq5.15","airq5.16", "airq5.17", "airq5.18", "airq5.20","airq5.21", "airq5.22", "airq5.23","airq5.26",
               "airq5.34", "airq5.35", "airq5.36","airq5.44", "airq5.45", "airq5.48","airq5.49", "airq5.50", "airq5.51","airq5.52",
               "airq5.53", "airq5.54", "airq5.56","airq5.57", "airq5.58", "airq5.59","airq5.60", "airq5.61", "airq5.62","airq5.63", "airq5.64", "airq5.65",
               "airq5.66", "airq5.67", "airq5.68","airq5.69", "airq5.70", "airq5.71"]
lightsen= ["light5.01", "light5.02", "light5.03","light5.04", "light5.05", "light5.06","light5.07", "light5.08", "light5.09", "light5.10", "light5.11", "light5.12",
               "light5.13", "light5.14", "light5.15","light5.16", "light5.17", "light5.18", "light5.20","light5.21", "light5.22", "light5.23","light5.26",
               "light5.34", "light5.35", "light5.36","light5.44", "light5.45", "light5.48","light5.49", "light5.50", "light5.51","light5.52",
               "light5.53", "light5.54", "light5.56","light5.57", "light5.58", "light5.59","light5.60", "light5.61", "light5.62","light5.63", "light5.64", "light5.65",
               "light5.66", "light5.67", "light5.68","light5.69", "light5.70", "light5.71"]
dustsen= ["dust5.01", "dust5.02", "dust5.03","dust5.04", "dust5.05", "dust5.06","dust5.07", "dust5.08", "dust5.09", "dust5.10", "dust5.11", "dust5.12",
               "dust5.13", "dust5.14", "dust5.15","dust5.16", "dust5.17", "dust5.18", "dust5.20","dust5.21", "dust5.22", "dust5.23","dust5.26",
               "dust5.34", "dust5.35", "dust5.36","dust5.44", "dust5.45", "dust5.48","dust5.49", "dust5.50", "dust5.51","dust5.52",
               "dust5.53", "dust5.54", "dust5.56","dust5.57", "dust5.58", "dust5.59","dust5.60", "dust5.61", "dust5.62","dust5.63", "dust5.64", "dust5.65",
               "dust5.66", "dust5.67", "dust5.68","dust5.69", "dust5.70", "dust5.71"]
pirsen= ["pir5.01", "pir5.02", "pir5.03","pir5.04", "pir5.05", "pir5.06","pir5.07", "pir5.08", "pir5.09", "pir5.10", "pir5.11", "pir5.12",
               "pir5.13", "pir5.14", "pir5.15","pir5.16", "pir5.17", "pir5.18", "pir5.20","pir5.21", "pir5.22", "pir5.23","pir5.26",
               "pir5.34", "pir5.35", "pir5.36","pir5.44", "pir5.45", "pir5.48","pir5.49", "pir5.50", "pir5.51","pir5.52",
               "pir5.53", "pir5.54", "pir5.56","pir5.57", "pir5.58", "pir5.59","pir5.60", "pir5.61", "pir5.62","pir5.63", "pir5.64", "pir5.65",
               "pir5.66", "pir5.67", "pir5.68","pir5.69", "pir5.70", "pir5.71"]
mqtwosen= ["mqtwo5.01", "mqtwo5.02", "mqtwo5.03","mqtwo5.04", "mqtwo5.05", "mqtwo5.06","mqtwo5.07", "mqtwo5.08", "mqtwo5.09", "mqtwo5.10", "mqtwo5.11", "mqtwo5.12",
               "mqtwo5.13", "mqtwo5.14", "mqtwo5.15","mqtwo5.16", "mqtwo5.17", "mqtwo5.18", "mqtwo5.20","mqtwo5.21", "mqtwo5.22", "mqtwo5.23","mqtwo5.26",
               "mqtwo5.34", "mqtwo5.35", "mqtwo5.36","mqtwo5.44", "mqtwo5.45", "mqtwo5.48","mqtwo5.49", "mqtwo5.50", "mqtwo5.51","mqtwo5.52",
               "mqtwo5.53", "mqtwo5.54", "mqtwo5.56","mqtwo5.57", "mqtwo5.58", "mqtwo5.59","mqtwo5.60", "mqtwo5.61", "mqtwo5.62","mqtwo5.63", "mqtwo5.64", "mqtwo5.65",
               "mqtwo5.66", "mqtwo5.67", "mqtwo5.68","mqtwo5.69", "mqtwo5.70", "mqtwo5.71"]
mqthreesen= ["mqthree5.01", "mqthree5.02", "mqthree5.03","mqthree5.04", "mqthree5.05", "mqthree5.06","mqthree5.07", "mqthree5.08", "mqthree5.09", "mqthree5.10", "mqthree5.11", "mqthree5.12",
               "mqthree5.13", "mqthree5.14", "mqthree5.15","mqthree5.16", "mqthree5.17", "mqthree5.18", "mqthree5.20","mqthree5.21", "mqthree5.22", "mqthree5.23","mqthree5.26",
               "mqthree5.34", "mqthree5.35", "mqthree5.36","mqthree5.44", "mqthree5.45", "mqthree5.48","mqthree5.49", "mqthree5.50", "mqthree5.51","mqthree5.52",
               "mqthree5.53", "mqthree5.54", "mqthree5.56","mqthree5.57", "mqthree5.58", "mqthree5.59","mqthree5.60", "mqthree5.61", "mqthree5.62","mqthree5.63", "mqthree5.64", "mqthree5.65",
               "mqthree5.66", "mqthree5.67", "mqthree5.68","mqthree5.69", "mqthree5.70", "mqthree5.71"]
mqfivesen= ["mqfive5.01", "mqfive5.02", "mqfive5.03","mqfive5.04", "mqfive5.05", "mqfive5.06","mqfive5.07", "mqfive5.08", "mqfive5.09", "mqfive5.10", "mqfive5.11", "mqfive5.12",
               "mqfive5.13", "mqfive5.14", "mqfive5.15","mqfive5.16", "mqfive5.17", "mqfive5.18", "mqfive5.20","mqfive5.21", "mqfive5.22", "mqfive5.23","mqfive5.26",
               "mqfive5.34", "mqfive5.35", "mqfive5.36","mqfive5.44", "mqfive5.45", "mqfive5.48","mqfive5.49", "mqfive5.50", "mqfive5.51","mqfive5.52",
               "mqfive5.53", "mqfive5.54", "mqfive5.56","mqfive5.57", "mqfive5.58", "mqfive5.59","mqfive5.60", "mqfive5.61", "mqfive5.62","mqfive5.63", "mqfive5.64", "mqfive5.65",
               "mqfive5.66", "mqfive5.67", "mqfive5.68","mqfive5.69", "mqfive5.70", "mqfive5.71"]
mqninesen= ["mqnine5.01", "mqnine5.02", "mqnine5.03","mqnine5.04", "mqnine5.05", "mqnine5.06","mqnine5.07", "mqnine5.08", "mqnine5.09", "mqnine5.10", "mqnine5.11", "mqnine5.12",
               "mqnine5.13", "mqnine5.14", "mqnine5.15","mqnine5.16", "mqnine5.17", "mqnine5.18", "mqnine5.20","mqnine5.21", "mqnine5.22", "mqnine5.23","mqnine5.26",
               "mqnine5.34", "mqnine5.35", "mqnine5.36","mqnine5.44", "mqnine5.45", "mqnine5.48","mqnine5.49", "mqnine5.50", "mqnine5.51","mqnine5.52",
               "mqnine5.53", "mqnine5.54", "mqnine5.56","mqnine5.57", "mqnine5.58", "mqnine5.59","mqnine5.60", "mqnine5.61", "mqnine5.62","mqnine5.63", "mqnine5.64", "mqnine5.65",
               "mqnine5.66", "mqnine5.67", "mqnine5.68","mqnine5.69", "mqnine5.70", "mqnine5.71"]
oxysen= ["oxy5.01", "oxy5.02", "oxy5.03","oxy5.04", "oxy5.05", "oxy5.06","oxy5.07", "oxy5.08", "oxy5.09", "oxy5.10", "oxy5.11", "oxy5.12",
               "oxy5.13", "oxy5.14", "oxy5.15","oxy5.16", "oxy5.17", "oxy5.18", "oxy5.20","oxy5.21", "oxy5.22", "oxy5.23","oxy5.26",
               "oxy5.34", "oxy5.35", "oxy5.36","oxy5.44", "oxy5.45", "oxy5.48","oxy5.49", "oxy5.50", "oxy5.51","oxy5.52",
               "oxy5.53", "oxy5.54", "oxy5.56","oxy5.57", "oxy5.58", "oxy5.59","oxy5.60", "oxy5.61", "oxy5.62","oxy5.63", "oxy5.64", "oxy5.65",
               "oxy5.66", "oxy5.67", "oxy5.68","oxy5.69", "oxy5.70", "oxy5.71"]
cotwosen= ["cotwo5.01", "cotwo5.02", "cotwo5.03","cotwo5.04", "cotwo5.05", "cotwo5.06","cotwo5.07", "cotwo5.08", "cotwo5.09", "cotwo5.10", "cotwo5.11", "cotwo5.12",
               "cotwo5.13", "cotwo5.14", "cotwo5.15","cotwo5.16", "cotwo5.17", "cotwo5.18", "cotwo5.20","cotwo5.21", "cotwo5.22", "cotwo5.23","cotwo5.26",
               "cotwo5.34", "cotwo5.35", "cotwo5.36","cotwo5.44", "cotwo5.45", "cotwo5.48","cotwo5.49", "cotwo5.50", "cotwo5.51","cotwo5.52",
               "cotwo5.53", "cotwo5.54", "cotwo5.56","cotwo5.57", "cotwo5.58", "cotwo5.59","cotwo5.60", "cotwo5.61", "cotwo5.62","cotwo5.63", "cotwo5.64", "cotwo5.65",
               "cotwo5.66", "cotwo5.67", "cotwo5.68","cotwo5.69", "cotwo5.70", "cotwo5.71"]
notwosen= ["notwo5.01", "notwo5.02", "notwo5.03","notwo5.04", "notwo5.05", "notwo5.06","notwo5.07", "notwo5.08", "notwo5.09", "notwo5.10", "notwo5.11", "notwo5.12",
               "notwo5.13", "notwo5.14", "notwo5.15","notwo5.16", "notwo5.17", "notwo5.18", "notwo5.20","notwo5.21", "notwo5.22", "notwo5.23","notwo5.26",
               "notwo5.34", "notwo5.35", "notwo5.36","notwo5.44", "notwo5.45", "notwo5.48","notwo5.49", "notwo5.50", "notwo5.51","notwo5.52",
               "notwo5.53", "notwo5.54", "notwo5.56","notwo5.57", "notwo5.58", "notwo5.59","notwo5.60", "notwo5.61", "notwo5.62","notwo5.63", "notwo5.64", "notwo5.65",
               "notwo5.66", "notwo5.67", "notwo5.68","notwo5.69", "notwo5.70", "notwo5.71"]
C2H5CHsen= ["C2H5CH5.01", "C2H5CH5.02", "C2H5CH5.03","C2H5CH5.04", "C2H5CH5.05", "C2H5CH5.06","C2H5CH5.07", "C2H5CH5.08", "C2H5CH5.09", "C2H5CH5.10", "C2H5CH5.11", "C2H5CH5.12",
               "C2H5CH5.13", "C2H5CH5.14", "C2H5CH5.15","C2H5CH5.16", "C2H5CH5.17", "C2H5CH5.18", "C2H5CH5.20","C2H5CH5.21", "C2H5CH5.22", "C2H5CH5.23","C2H5CH5.26",
               "C2H5CH5.34", "C2H5CH5.35", "C2H5CH5.36","C2H5CH5.44", "C2H5CH5.45", "C2H5CH5.48","C2H5CH5.49", "C2H5CH5.50", "C2H5CH5.51","C2H5CH5.52",
               "C2H5CH5.53", "C2H5CH5.54", "C2H5CH5.56","C2H5CH5.57", "C2H5CH5.58", "C2H5CH5.59","C2H5CH5.60", "C2H5CH5.61", "C2H5CH5.62","C2H5CH5.63", "C2H5CH5.64", "C2H5CH5.65",
               "C2H5CH5.66", "C2H5CH5.67", "C2H5CH5.68","C2H5CH5.69", "C2H5CH5.70", "C2H5CH5.71"]
vocsen= ["voc5.01", "voc5.02", "voc5.03","voc5.04", "voc5.05", "voc5.06","voc5.07", "voc5.08", "voc5.09", "voc5.10", "voc5.11", "voc5.12",
               "voc5.13", "voc5.14", "voc5.15","voc5.16", "voc5.17", "voc5.18", "voc5.20","voc5.21", "voc5.22", "voc5.23","voc5.26",
               "voc5.34", "voc5.35", "voc5.36","voc5.44", "voc5.45", "voc5.48","voc5.49", "voc5.50", "voc5.51","voc5.52",
               "voc5.53", "voc5.54", "voc5.56","voc5.57", "voc5.58", "voc5.59","voc5.60", "voc5.61", "voc5.62","voc5.63", "voc5.64", "voc5.65",
               "voc5.66", "voc5.67", "voc5.68","voc5.69", "voc5.70", "voc5.71"]
cosen= ["co5.01", "co5.02", "co5.03","co5.04", "co5.05", "co5.06","co5.07", "co5.08", "co5.09", "co5.10", "co5.11", "co5.12",
               "co5.13", "co5.14", "co5.15","co5.16", "co5.17", "co5.18", "co5.20","co5.21", "co5.22", "co5.23","co5.26",
               "co5.34", "co5.35", "co5.36","co5.44", "co5.45", "co5.48","co5.49", "co5.50", "co5.51","co5.52",
               "co5.53", "co5.54", "co5.56","co5.57", "co5.58", "co5.59","co5.60", "co5.61", "co5.62","co5.63", "co5.64", "co5.65",
               "co5.66", "co5.67", "co5.68","co5.69", "co5.70", "co5.71"]



In [9]:
for i in tempsen:
        g.add((bldg[tempsen], A, BRICK["Temperature_Sensor"]))
        g.add((bldg[tempsen], RDFS.label, Literal("Room Temperature Sensor")))
for i in humsen:
        g.add((bldg[humsen], A, BRICK["Humidity_Sensor"]))
        g.add((bldg[humsen], RDFS.label, Literal("Room Humidity Sensor")))
for i in soundsen:
        g.add((bldg[soundsen], A, BRICK["Sensor"]))
        g.add((bldg[soundsen], RDFS.label, Literal("Room Sound Sensor")))
for i in hchosen:
        g.add((bldg[hchosen], A, BRICK["Formaldehyde_Sensor"]))
        g.add((bldg[hchosen], RDFS.label, Literal("Room hcho/Formaldehyde Sensor")))
for i in airqsen:
        g.add((bldg[airqsen], A, BRICK["Air_Quality_Sensor"]))
        g.add((bldg[airqsen], RDFS.label, Literal("Room Air Quality Sensor")))
for i in lightsen:
        g.add((bldg[lightsen], A, BRICK["Illuminance_Sensor"]))
        g.add((bldg[lightsen], RDFS.label, Literal("Room Light Sensor")))
for i in dustsen:
        g.add((bldg[dustsen], A, BRICK["PM2.5_Sensor"]))
        g.add((bldg[dustsen], RDFS.label, Literal("Room pm 2.5 Dust Sensor")))
for i in pirsen:
        g.add((bldg[pirsen], A, BRICK["PIR_Sensor"]))
        g.add((bldg[pirsen], RDFS.label, Literal("Room Occupancy Sensor")))
for i in mqtwosen:
        g.add((bldg[mqtwosen], A, BRICK["Gas_Sensor"]))
        g.add((bldg[mqtwosen], RDFS.label, Literal("Room MQ2 Gas Sensor")))
for i in mqthreesen:
        g.add((bldg[mqthreesen], A, BRICK["Gas_Sensor"]))
        g.add((bldg[mqthreesen], RDFS.label, Literal("Room MQ3 Gas Sensor")))
for i in mqfivesen:
        g.add((bldg[mqfivesen], A, BRICK["Gas_Sensor"]))
        g.add((bldg[mqfivesen], RDFS.label, Literal("Room MQ5 Gas Sensor")))
for i in mqninesen:
        g.add((bldg[mqninesen], A, BRICK["Gas_Sensor"]))
        g.add((bldg[mqninesen], RDFS.label, Literal("Room MQ9 Gas Sensor")))
for i in oxysen:
        g.add((bldg[oxysen], A, BRICK["Gas_Sensor"]))
        g.add((bldg[oxysen], RDFS.label, Literal("Room Oxygen gas Sensor")))
for i in cotwosen:
        g.add((bldg[cotwosen], A, BRICK["CO2_Sensor"]))
        g.add((bldg[cotwosen], RDFS.label, Literal("Room CO2 Sensor")))
for i in notwosen:
        g.add((bldg[notwosen], A, BRICK["NO2_Level_Sensor"]))
        g.add((bldg[notwosen], RDFS.label, Literal("Room NO2 Sensor")))
for i in C2H5CHsen:
        g.add((bldg[C2H5CHsen], A, BRICK["Gas_Sensor"]))
        g.add((bldg[C2H5CHsen], RDFS.label, Literal("Room C2H5CH Sensor")))
for i in vocsen:
        g.add((bldg[vocsen], A, BRICK["TVOC_Level_Sensor"]))
        g.add((bldg[vocsen], RDFS.label, Literal("Room VOC Sensor")))
for i in cosen:
        g.add((bldg[cosen], A, BRICK["CO_Sensor"]))
        g.add((bldg[cosen], RDFS.label, Literal("Room CO Sensor")))                

In [12]:
# g.serialize("abacws-building-v2.ttl", format="ttl")


<Graph identifier=N9c08d45750fc47b1938240b2d60d2f54 (<class 'brickschema.graph.Graph'>)>

In [10]:
temp_and_rooms = {
"temp5.01":"Room5.01", "temp5.02":"Room5.02","temp5.03":"Room5.03","temp5.04":"Room5.04","temp5.05":"Room5.05","temp5.06":"Room5.06",
"temp5.07":"Room5.07", "temp5.08":"Room5.08","temp5.09":"Room5.09", "temp5.10":"Room5.10", "temp5.11":"Room5.11", "temp5.12":"Room5.12",
"temp5.13":"Room5.13","temp5.14":"Room5.14", "temp5.15":"Room5.15","temp5.16":"Room5.16","temp5.17":"Room5.17","temp5.18":"Room5.18",
"temp5.20":"Room5.20","temp5.21":"Room5.21", "temp5.22":"Room5.22", "temp5.23":"Room5.23","temp5.26":"Room5.26","temp5.34":"Room5.34",
"temp5.35":"Room5.35", "temp5.36":"Room5.36","temp5.44":"Room5.44","temp5.45":"Room5.45","temp5.48":"Room5.48","temp5.49":"Room5.49",
"temp5.50":"Room5.50", "temp5.51":"Room5.51","temp5.52":"Room5.52", "temp5.53":"Room5.53", "temp5.54":"Room5.54", "temp5.56":"Room5.56",
"temp5.57":"Room5.57", "temp5.58":"Room5.58", "temp5.59":"Room5.59","temp5.60":"Room5.60", "temp5.61":"Room5.61", "temp5.62":"Room5.62",
"temp5.63":"Room5.63", "temp5.64":"Room5.64", "temp5.65":"Room5.65","temp5.66":"Room5.66", "temp5.67":"Room5.67", "temp5.68":"Room5.68",
"temp5.69":"Room5.69", "temp5.70":"Room5.70", "temp5.71":"Room5.71"
}
hum_and_rooms = {
"hum5.01":"Room5.01", "hum5.02":"Room5.02","hum5.03":"Room5.03","hum5.04":"Room5.04","hum5.05":"Room5.05","hum5.06":"Room5.06",
"hum5.07":"Room5.07", "hum5.08":"Room5.08","hum5.09":"Room5.09", "hum5.10":"Room5.10", "hum5.11":"Room5.11", "hum5.12":"Room5.12",
"hum5.13":"Room5.13","hum5.14":"Room5.14", "hum5.15":"Room5.15","hum5.16":"Room5.16","hum5.17":"Room5.17","hum5.18":"Room5.18",
"hum5.20":"Room5.20","hum5.21":"Room5.21", "hum5.22":"Room5.22", "hum5.23":"Room5.23","hum5.26":"Room5.26","hum5.34":"Room5.34",
"hum5.35":"Room5.35", "hum5.36":"Room5.36","hum5.44":"Room5.44","hum5.45":"Room5.45","hum5.48":"Room5.48","hum5.49":"Room5.49",
"hum5.50":"Room5.50", "hum5.51":"Room5.51","hum5.52":"Room5.52", "hum5.53":"Room5.53", "hum5.54":"Room5.54", "hum5.56":"Room5.56",
"hum5.57":"Room5.57", "hum5.58":"Room5.58", "hum5.59":"Room5.59","hum5.60":"Room5.60", "hum5.61":"Room5.61", "hum5.62":"Room5.62",
"hum5.63":"Room5.63", "hum5.64":"Room5.64", "hum5.65":"Room5.65","hum5.66":"Room5.66", "hum5.67":"Room5.67", "hum5.68":"Room5.68",
"hum5.69":"Room5.69", "hum5.70":"Room5.70", "hum5.71":"Room5.71"
}
sound_and_rooms = {
"sound5.01":"Room5.01", "sound5.02":"Room5.02","sound5.03":"Room5.03","sound5.04":"Room5.04","sound5.05":"Room5.05","sound5.06":"Room5.06",
"sound5.07":"Room5.07", "sound5.08":"Room5.08","sound5.09":"Room5.09", "sound5.10":"Room5.10", "sound5.11":"Room5.11", "sound5.12":"Room5.12",
"sound5.13":"Room5.13","sound5.14":"Room5.14", "sound5.15":"Room5.15","sound5.16":"Room5.16","sound5.17":"Room5.17","sound5.18":"Room5.18",
"sound5.20":"Room5.20","sound5.21":"Room5.21", "sound5.22":"Room5.22", "sound5.23":"Room5.23","sound5.26":"Room5.26","sound5.34":"Room5.34",
"sound5.35":"Room5.35", "sound5.36":"Room5.36","sound5.44":"Room5.44","sound5.45":"Room5.45","sound5.48":"Room5.48","sound5.49":"Room5.49",
"sound5.50":"Room5.50", "sound5.51":"Room5.51","sound5.52":"Room5.52", "sound5.53":"Room5.53", "sound5.54":"Room5.54", "sound5.56":"Room5.56",
"sound5.57":"Room5.57", "sound5.58":"Room5.58", "sound5.59":"Room5.59","sound5.60":"Room5.60", "sound5.61":"Room5.61", "sound5.62":"Room5.62",
"sound5.63":"Room5.63", "sound5.64":"Room5.64", "sound5.65":"Room5.65","sound5.66":"Room5.66", "sound5.67":"Room5.67", "sound5.68":"Room5.68",
"sound5.69":"Room5.69", "sound5.70":"Room5.70", "sound5.71":"Room5.71"
}
hcho_and_rooms = {
"hcho5.01":"Room5.01", "hcho5.02":"Room5.02","hcho5.03":"Room5.03","hcho5.04":"Room5.04","hcho5.05":"Room5.05","hcho5.06":"Room5.06",
"hcho5.07":"Room5.07", "hcho5.08":"Room5.08","hcho5.09":"Room5.09", "hcho5.10":"Room5.10", "hcho5.11":"Room5.11", "hcho5.12":"Room5.12",
"hcho5.13":"Room5.13","hcho5.14":"Room5.14", "hcho5.15":"Room5.15","hcho5.16":"Room5.16","hcho5.17":"Room5.17","hcho5.18":"Room5.18",
"hcho5.20":"Room5.20","hcho5.21":"Room5.21", "hcho5.22":"Room5.22", "hcho5.23":"Room5.23","hcho5.26":"Room5.26","hcho5.34":"Room5.34",
"hcho5.35":"Room5.35", "hcho5.36":"Room5.36","hcho5.44":"Room5.44","hcho5.45":"Room5.45","hcho5.48":"Room5.48","hcho5.49":"Room5.49",
"hcho5.50":"Room5.50", "hcho5.51":"Room5.51","hcho5.52":"Room5.52", "hcho5.53":"Room5.53", "hcho5.54":"Room5.54", "hcho5.56":"Room5.56",
"hcho5.57":"Room5.57", "hcho5.58":"Room5.58", "hcho5.59":"Room5.59","hcho5.60":"Room5.60", "hcho5.61":"Room5.61", "hcho5.62":"Room5.62",
"hcho5.63":"Room5.63", "hcho5.64":"Room5.64", "hcho5.65":"Room5.65","hcho5.66":"Room5.66", "hcho5.67":"Room5.67", "hcho5.68":"Room5.68",
"hcho5.69":"Room5.69", "hcho5.70":"Room5.70", "hcho5.71":"Room5.71"
}
airq_and_rooms = {
"airq5.01":"Room5.01", "airq5.02":"Room5.02","airq5.03":"Room5.03","airq5.04":"Room5.04","airq5.05":"Room5.05","airq5.06":"Room5.06",
"airq5.07":"Room5.07", "airq5.08":"Room5.08","airq5.09":"Room5.09", "airq5.10":"Room5.10", "airq5.11":"Room5.11", "airq5.12":"Room5.12",
"airq5.13":"Room5.13","airq5.14":"Room5.14", "airq5.15":"Room5.15","airq5.16":"Room5.16","airq5.17":"Room5.17","airq5.18":"Room5.18",
"airq5.20":"Room5.20","airq5.21":"Room5.21", "airq5.22":"Room5.22", "airq5.23":"Room5.23","airq5.26":"Room5.26","airq5.34":"Room5.34",
"airq5.35":"Room5.35", "airq5.36":"Room5.36","airq5.44":"Room5.44","airq5.45":"Room5.45","airq5.48":"Room5.48","airq5.49":"Room5.49",
"airq5.50":"Room5.50", "airq5.51":"Room5.51","airq5.52":"Room5.52", "airq5.53":"Room5.53", "airq5.54":"Room5.54", "airq5.56":"Room5.56",
"airq5.57":"Room5.57", "airq5.58":"Room5.58", "airq5.59":"Room5.59","airq5.60":"Room5.60", "airq5.61":"Room5.61", "airq5.62":"Room5.62",
"airq5.63":"Room5.63", "airq5.64":"Room5.64", "airq5.65":"Room5.65","airq5.66":"Room5.66", "airq5.67":"Room5.67", "airq5.68":"Room5.68",
"airq5.69":"Room5.69", "airq5.70":"Room5.70", "airq5.71":"Room5.71"
}
light_and_rooms = {
"light5.01":"Room5.01", "light5.02":"Room5.02","light5.03":"Room5.03","light5.04":"Room5.04","light5.05":"Room5.05","light5.06":"Room5.06",
"light5.07":"Room5.07", "light5.08":"Room5.08","light5.09":"Room5.09", "light5.10":"Room5.10", "light5.11":"Room5.11", "light5.12":"Room5.12",
"light5.13":"Room5.13","light5.14":"Room5.14", "light5.15":"Room5.15","light5.16":"Room5.16","light5.17":"Room5.17","light5.18":"Room5.18",
"light5.20":"Room5.20","light5.21":"Room5.21", "light5.22":"Room5.22", "light5.23":"Room5.23","light5.26":"Room5.26","light5.34":"Room5.34",
"light5.35":"Room5.35", "light5.36":"Room5.36","light5.44":"Room5.44","light5.45":"Room5.45","light5.48":"Room5.48","light5.49":"Room5.49",
"light5.50":"Room5.50", "light5.51":"Room5.51","light5.52":"Room5.52", "light5.53":"Room5.53", "light5.54":"Room5.54", "light5.56":"Room5.56",
"light5.57":"Room5.57", "light5.58":"Room5.58", "light5.59":"Room5.59","light5.60":"Room5.60", "light5.61":"Room5.61", "light5.62":"Room5.62",
"light5.63":"Room5.63", "light5.64":"Room5.64", "light5.65":"Room5.65","light5.66":"Room5.66", "light5.67":"Room5.67", "light5.68":"Room5.68",
"light5.69":"Room5.69", "light5.70":"Room5.70", "light5.71":"Room5.71"
}
dust_and_rooms = {
"dust5.01":"Room5.01", "dust5.02":"Room5.02","dust5.03":"Room5.03","dust5.04":"Room5.04","dust5.05":"Room5.05","dust5.06":"Room5.06",
"dust5.07":"Room5.07", "dust5.08":"Room5.08","dust5.09":"Room5.09", "dust5.10":"Room5.10", "dust5.11":"Room5.11", "dust5.12":"Room5.12",
"dust5.13":"Room5.13","dust5.14":"Room5.14", "dust5.15":"Room5.15","dust5.16":"Room5.16","dust5.17":"Room5.17","dust5.18":"Room5.18",
"dust5.20":"Room5.20","dust5.21":"Room5.21", "dust5.22":"Room5.22", "dust5.23":"Room5.23","dust5.26":"Room5.26","dust5.34":"Room5.34",
"dust5.35":"Room5.35", "dust5.36":"Room5.36","dust5.44":"Room5.44","dust5.45":"Room5.45","dust5.48":"Room5.48","dust5.49":"Room5.49",
"dust5.50":"Room5.50", "dust5.51":"Room5.51","dust5.52":"Room5.52", "dust5.53":"Room5.53", "dust5.54":"Room5.54", "dust5.56":"Room5.56",
"dust5.57":"Room5.57", "dust5.58":"Room5.58", "dust5.59":"Room5.59","dust5.60":"Room5.60", "dust5.61":"Room5.61", "dust5.62":"Room5.62",
"dust5.63":"Room5.63", "dust5.64":"Room5.64", "dust5.65":"Room5.65","dust5.66":"Room5.66", "dust5.67":"Room5.67", "dust5.68":"Room5.68",
"dust5.69":"Room5.69", "dust5.70":"Room5.70", "dust5.71":"Room5.71"
}
pir_and_rooms = {
"pir5.01":"Room5.01", "pir5.02":"Room5.02","pir5.03":"Room5.03","pir5.04":"Room5.04","pir5.05":"Room5.05","pir5.06":"Room5.06",
"pir5.07":"Room5.07", "pir5.08":"Room5.08","pir5.09":"Room5.09", "pir5.10":"Room5.10", "pir5.11":"Room5.11", "pir5.12":"Room5.12",
"pir5.13":"Room5.13","pir5.14":"Room5.14", "pir5.15":"Room5.15","pir5.16":"Room5.16","pir5.17":"Room5.17","pir5.18":"Room5.18",
"pir5.20":"Room5.20","pir5.21":"Room5.21", "pir5.22":"Room5.22", "pir5.23":"Room5.23","pir5.26":"Room5.26","pir5.34":"Room5.34",
"pir5.35":"Room5.35", "pir5.36":"Room5.36","pir5.44":"Room5.44","pir5.45":"Room5.45","pir5.48":"Room5.48","pir5.49":"Room5.49",
"pir5.50":"Room5.50", "pir5.51":"Room5.51","pir5.52":"Room5.52", "pir5.53":"Room5.53", "pir5.54":"Room5.54", "pir5.56":"Room5.56",
"pir5.57":"Room5.57", "pir5.58":"Room5.58", "pir5.59":"Room5.59","pir5.60":"Room5.60", "pir5.61":"Room5.61", "pir5.62":"Room5.62",
"pir5.63":"Room5.63", "pir5.64":"Room5.64", "pir5.65":"Room5.65","pir5.66":"Room5.66", "pir5.67":"Room5.67", "pir5.68":"Room5.68",
"pir5.69":"Room5.69", "pir5.70":"Room5.70", "pir5.71":"Room5.71"
}
mqtwo_and_rooms = {
"mqtwo5.01":"Room5.01", "mqtwo5.02":"Room5.02","mqtwo5.03":"Room5.03","mqtwo5.04":"Room5.04","mqtwo5.05":"Room5.05","mqtwo5.06":"Room5.06",
"mqtwo5.07":"Room5.07", "mqtwo5.08":"Room5.08","mqtwo5.09":"Room5.09", "mqtwo5.10":"Room5.10", "mqtwo5.11":"Room5.11", "mqtwo5.12":"Room5.12",
"mqtwo5.13":"Room5.13","mqtwo5.14":"Room5.14", "mqtwo5.15":"Room5.15","mqtwo5.16":"Room5.16","mqtwo5.17":"Room5.17","mqtwo5.18":"Room5.18",
"mqtwo5.20":"Room5.20","mqtwo5.21":"Room5.21", "mqtwo5.22":"Room5.22", "mqtwo5.23":"Room5.23","mqtwo5.26":"Room5.26","mqtwo5.34":"Room5.34",
"mqtwo5.35":"Room5.35", "mqtwo5.36":"Room5.36","mqtwo5.44":"Room5.44","mqtwo5.45":"Room5.45","mqtwo5.48":"Room5.48","mqtwo5.49":"Room5.49",
"mqtwo5.50":"Room5.50", "mqtwo5.51":"Room5.51","mqtwo5.52":"Room5.52", "mqtwo5.53":"Room5.53", "mqtwo5.54":"Room5.54", "mqtwo5.56":"Room5.56",
"mqtwo5.57":"Room5.57", "mqtwo5.58":"Room5.58", "mqtwo5.59":"Room5.59","mqtwo5.60":"Room5.60", "mqtwo5.61":"Room5.61", "mqtwo5.62":"Room5.62",
"mqtwo5.63":"Room5.63", "mqtwo5.64":"Room5.64", "mqtwo5.65":"Room5.65","mqtwo5.66":"Room5.66", "mqtwo5.67":"Room5.67", "mqtwo5.68":"Room5.68",
"mqtwo5.69":"Room5.69", "mqtwo5.70":"Room5.70", "mqtwo5.71":"Room5.71"
}
mqthree_and_rooms = {
"mqthree5.01":"Room5.01", "mqthree5.02":"Room5.02","mqthree5.03":"Room5.03","mqthree5.04":"Room5.04","mqthree5.05":"Room5.05","mqthree5.06":"Room5.06",
"mqthree5.07":"Room5.07", "mqthree5.08":"Room5.08","mqthree5.09":"Room5.09", "mqthree5.10":"Room5.10", "mqthree5.11":"Room5.11", "mqthree5.12":"Room5.12",
"mqthree5.13":"Room5.13","mqthree5.14":"Room5.14", "mqthree5.15":"Room5.15","mqthree5.16":"Room5.16","mqthree5.17":"Room5.17","mqthree5.18":"Room5.18",
"mqthree5.20":"Room5.20","mqthree5.21":"Room5.21", "mqthree5.22":"Room5.22", "mqthree5.23":"Room5.23","mqthree5.26":"Room5.26","mqthree5.34":"Room5.34",
"mqthree5.35":"Room5.35", "mqthree5.36":"Room5.36","mqthree5.44":"Room5.44","mqthree5.45":"Room5.45","mqthree5.48":"Room5.48","mqthree5.49":"Room5.49",
"mqthree5.50":"Room5.50", "mqthree5.51":"Room5.51","mqthree5.52":"Room5.52", "mqthree5.53":"Room5.53", "mqthree5.54":"Room5.54", "mqthree5.56":"Room5.56",
"mqthree5.57":"Room5.57", "mqthree5.58":"Room5.58", "mqthree5.59":"Room5.59","mqthree5.60":"Room5.60", "mqthree5.61":"Room5.61", "mqthree5.62":"Room5.62",
"mqthree5.63":"Room5.63", "mqthree5.64":"Room5.64", "mqthree5.65":"Room5.65","mqthree5.66":"Room5.66", "mqthree5.67":"Room5.67", "mqthree5.68":"Room5.68",
"mqthree5.69":"Room5.69", "mqthree5.70":"Room5.70", "mqthree5.71":"Room5.71"
}
mqfive_and_rooms = {
"mqfive5.01":"Room5.01", "mqfive5.02":"Room5.02","mqfive5.03":"Room5.03","mqfive5.04":"Room5.04","mqfive5.05":"Room5.05","mqfive5.06":"Room5.06",
"mqfive5.07":"Room5.07", "mqfive5.08":"Room5.08","mqfive5.09":"Room5.09", "mqfive5.10":"Room5.10", "mqfive5.11":"Room5.11", "mqfive5.12":"Room5.12",
"mqfive5.13":"Room5.13","mqfive5.14":"Room5.14", "mqfive5.15":"Room5.15","mqfive5.16":"Room5.16","mqfive5.17":"Room5.17","mqfive5.18":"Room5.18",
"mqfive5.20":"Room5.20","mqfive5.21":"Room5.21", "mqfive5.22":"Room5.22", "mqfive5.23":"Room5.23","mqfive5.26":"Room5.26","mqfive5.34":"Room5.34",
"mqfive5.35":"Room5.35", "mqfive5.36":"Room5.36","mqfive5.44":"Room5.44","mqfive5.45":"Room5.45","mqfive5.48":"Room5.48","mqfive5.49":"Room5.49",
"mqfive5.50":"Room5.50", "mqfive5.51":"Room5.51","mqfive5.52":"Room5.52", "mqfive5.53":"Room5.53", "mqfive5.54":"Room5.54", "mqfive5.56":"Room5.56",
"mqfive5.57":"Room5.57", "mqfive5.58":"Room5.58", "mqfive5.59":"Room5.59","mqfive5.60":"Room5.60", "mqfive5.61":"Room5.61", "mqfive5.62":"Room5.62",
"mqfive5.63":"Room5.63", "mqfive5.64":"Room5.64", "mqfive5.65":"Room5.65","mqfive5.66":"Room5.66", "mqfive5.67":"Room5.67", "mqfive5.68":"Room5.68",
"mqfive5.69":"Room5.69", "mqfive5.70":"Room5.70", "mqfive5.71":"Room5.71"
}
mqnine_and_rooms = {
"mqnine5.01":"Room5.01", "mqnine5.02":"Room5.02","mqnine5.03":"Room5.03","mqnine5.04":"Room5.04","mqnine5.05":"Room5.05","mqnine5.06":"Room5.06",
"mqnine5.07":"Room5.07", "mqnine5.08":"Room5.08","mqnine5.09":"Room5.09", "mqnine5.10":"Room5.10", "mqnine5.11":"Room5.11", "mqnine5.12":"Room5.12",
"mqnine5.13":"Room5.13","mqnine5.14":"Room5.14", "mqnine5.15":"Room5.15","mqnine5.16":"Room5.16","mqnine5.17":"Room5.17","mqnine5.18":"Room5.18",
"mqnine5.20":"Room5.20","mqnine5.21":"Room5.21", "mqnine5.22":"Room5.22", "mqnine5.23":"Room5.23","mqnine5.26":"Room5.26","mqnine5.34":"Room5.34",
"mqnine5.35":"Room5.35", "mqnine5.36":"Room5.36","mqnine5.44":"Room5.44","mqnine5.45":"Room5.45","mqnine5.48":"Room5.48","mqnine5.49":"Room5.49",
"mqnine5.50":"Room5.50", "mqnine5.51":"Room5.51","mqnine5.52":"Room5.52", "mqnine5.53":"Room5.53", "mqnine5.54":"Room5.54", "mqnine5.56":"Room5.56",
"mqnine5.57":"Room5.57", "mqnine5.58":"Room5.58", "mqnine5.59":"Room5.59","mqnine5.60":"Room5.60", "mqnine5.61":"Room5.61", "mqnine5.62":"Room5.62",
"mqnine5.63":"Room5.63", "mqnine5.64":"Room5.64", "mqnine5.65":"Room5.65","mqnine5.66":"Room5.66", "mqnine5.67":"Room5.67", "mqnine5.68":"Room5.68",
"mqnine5.69":"Room5.69", "mqnine5.70":"Room5.70", "mqnine5.71":"Room5.71"
}
oxy_and_rooms = {
"oxy5.01":"Room5.01", "oxy5.02":"Room5.02","oxy5.03":"Room5.03","oxy5.04":"Room5.04","oxy5.05":"Room5.05","oxy5.06":"Room5.06",
"oxy5.07":"Room5.07", "oxy5.08":"Room5.08","oxy5.09":"Room5.09", "oxy5.10":"Room5.10", "oxy5.11":"Room5.11", "oxy5.12":"Room5.12",
"oxy5.13":"Room5.13","oxy5.14":"Room5.14", "oxy5.15":"Room5.15","oxy5.16":"Room5.16","oxy5.17":"Room5.17","oxy5.18":"Room5.18",
"oxy5.20":"Room5.20","oxy5.21":"Room5.21", "oxy5.22":"Room5.22", "oxy5.23":"Room5.23","oxy5.26":"Room5.26","oxy5.34":"Room5.34",
"oxy5.35":"Room5.35", "oxy5.36":"Room5.36","oxy5.44":"Room5.44","oxy5.45":"Room5.45","oxy5.48":"Room5.48","oxy5.49":"Room5.49",
"oxy5.50":"Room5.50", "oxy5.51":"Room5.51","oxy5.52":"Room5.52", "oxy5.53":"Room5.53", "oxy5.54":"Room5.54", "oxy5.56":"Room5.56",
"oxy5.57":"Room5.57", "oxy5.58":"Room5.58", "oxy5.59":"Room5.59","oxy5.60":"Room5.60", "oxy5.61":"Room5.61", "oxy5.62":"Room5.62",
"oxy5.63":"Room5.63", "oxy5.64":"Room5.64", "oxy5.65":"Room5.65","oxy5.66":"Room5.66", "oxy5.67":"Room5.67", "oxy5.68":"Room5.68",
"oxy5.69":"Room5.69", "oxy5.70":"Room5.70", "oxy5.71":"Room5.71"
}
cotwo_and_rooms = {
"cotwo5.01":"Room5.01", "cotwo5.02":"Room5.02","cotwo5.03":"Room5.03","cotwo5.04":"Room5.04","cotwo5.05":"Room5.05","cotwo5.06":"Room5.06",
"cotwo5.07":"Room5.07", "cotwo5.08":"Room5.08","cotwo5.09":"Room5.09", "cotwo5.10":"Room5.10", "cotwo5.11":"Room5.11", "cotwo5.12":"Room5.12",
"cotwo5.13":"Room5.13","cotwo5.14":"Room5.14", "cotwo5.15":"Room5.15","cotwo5.16":"Room5.16","cotwo5.17":"Room5.17","cotwo5.18":"Room5.18",
"cotwo5.20":"Room5.20","cotwo5.21":"Room5.21", "cotwo5.22":"Room5.22", "cotwo5.23":"Room5.23","cotwo5.26":"Room5.26","cotwo5.34":"Room5.34",
"cotwo5.35":"Room5.35", "cotwo5.36":"Room5.36","cotwo5.44":"Room5.44","cotwo5.45":"Room5.45","cotwo5.48":"Room5.48","cotwo5.49":"Room5.49",
"cotwo5.50":"Room5.50", "cotwo5.51":"Room5.51","cotwo5.52":"Room5.52", "cotwo5.53":"Room5.53", "cotwo5.54":"Room5.54", "cotwo5.56":"Room5.56",
"cotwo5.57":"Room5.57", "cotwo5.58":"Room5.58", "cotwo5.59":"Room5.59","cotwo5.60":"Room5.60", "cotwo5.61":"Room5.61", "cotwo5.62":"Room5.62",
"cotwo5.63":"Room5.63", "cotwo5.64":"Room5.64", "cotwo5.65":"Room5.65","cotwo5.66":"Room5.66", "cotwo5.67":"Room5.67", "cotwo5.68":"Room5.68",
"cotwo5.69":"Room5.69", "cotwo5.70":"Room5.70", "cotwo5.71":"Room5.71"
}
notwo_and_rooms = {
"notwo5.01":"Room5.01", "notwo5.02":"Room5.02","notwo5.03":"Room5.03","notwo5.04":"Room5.04","notwo5.05":"Room5.05","notwo5.06":"Room5.06",
"notwo5.07":"Room5.07", "notwo5.08":"Room5.08","notwo5.09":"Room5.09", "notwo5.10":"Room5.10", "notwo5.11":"Room5.11", "notwo5.12":"Room5.12",
"notwo5.13":"Room5.13","notwo5.14":"Room5.14", "notwo5.15":"Room5.15","notwo5.16":"Room5.16","notwo5.17":"Room5.17","notwo5.18":"Room5.18",
"notwo5.20":"Room5.20","notwo5.21":"Room5.21", "notwo5.22":"Room5.22", "notwo5.23":"Room5.23","notwo5.26":"Room5.26","notwo5.34":"Room5.34",
"notwo5.35":"Room5.35", "notwo5.36":"Room5.36","notwo5.44":"Room5.44","notwo5.45":"Room5.45","notwo5.48":"Room5.48","notwo5.49":"Room5.49",
"notwo5.50":"Room5.50", "notwo5.51":"Room5.51","notwo5.52":"Room5.52", "notwo5.53":"Room5.53", "notwo5.54":"Room5.54", "notwo5.56":"Room5.56",
"notwo5.57":"Room5.57", "notwo5.58":"Room5.58", "notwo5.59":"Room5.59","notwo5.60":"Room5.60", "notwo5.61":"Room5.61", "notwo5.62":"Room5.62",
"notwo5.63":"Room5.63", "notwo5.64":"Room5.64", "notwo5.65":"Room5.65","notwo5.66":"Room5.66", "notwo5.67":"Room5.67", "notwo5.68":"Room5.68",
"notwo5.69":"Room5.69", "notwo5.70":"Room5.70", "notwo5.71":"Room5.71"
}
c2h5ch_and_rooms = {
"c2h5ch5.01":"Room5.01", "c2h5ch5.02":"Room5.02","c2h5ch5.03":"Room5.03","c2h5ch5.04":"Room5.04","c2h5ch5.05":"Room5.05","c2h5ch5.06":"Room5.06",
"c2h5ch5.07":"Room5.07", "c2h5ch5.08":"Room5.08","c2h5ch5.09":"Room5.09", "c2h5ch5.10":"Room5.10", "c2h5ch5.11":"Room5.11", "c2h5ch5.12":"Room5.12",
"c2h5ch5.13":"Room5.13","c2h5ch5.14":"Room5.14", "c2h5ch5.15":"Room5.15","c2h5ch5.16":"Room5.16","c2h5ch5.17":"Room5.17","c2h5ch5.18":"Room5.18",
"c2h5ch5.20":"Room5.20","c2h5ch5.21":"Room5.21", "c2h5ch5.22":"Room5.22", "c2h5ch5.23":"Room5.23","c2h5ch5.26":"Room5.26","c2h5ch5.34":"Room5.34",
"c2h5ch5.35":"Room5.35", "c2h5ch5.36":"Room5.36","c2h5ch5.44":"Room5.44","c2h5ch5.45":"Room5.45","c2h5ch5.48":"Room5.48","c2h5ch5.49":"Room5.49",
"c2h5ch5.50":"Room5.50", "c2h5ch5.51":"Room5.51","c2h5ch5.52":"Room5.52", "c2h5ch5.53":"Room5.53", "c2h5ch5.54":"Room5.54", "c2h5ch5.56":"Room5.56",
"c2h5ch5.57":"Room5.57", "c2h5ch5.58":"Room5.58", "c2h5ch5.59":"Room5.59","c2h5ch5.60":"Room5.60", "c2h5ch5.61":"Room5.61", "c2h5ch5.62":"Room5.62",
"c2h5ch5.63":"Room5.63", "c2h5ch5.64":"Room5.64", "c2h5ch5.65":"Room5.65","c2h5ch5.66":"Room5.66", "c2h5ch5.67":"Room5.67", "c2h5ch5.68":"Room5.68",
"c2h5ch5.69":"Room5.69", "c2h5ch5.70":"Room5.70", "c2h5ch5.71":"Room5.71"
}
voc_and_rooms = {
"voc5.01":"Room5.01", "voc5.02":"Room5.02","voc5.03":"Room5.03","voc5.04":"Room5.04","voc5.05":"Room5.05","voc5.06":"Room5.06",
"voc5.07":"Room5.07", "voc5.08":"Room5.08","voc5.09":"Room5.09", "voc5.10":"Room5.10", "voc5.11":"Room5.11", "voc5.12":"Room5.12",
"voc5.13":"Room5.13","voc5.14":"Room5.14", "voc5.15":"Room5.15","voc5.16":"Room5.16","voc5.17":"Room5.17","voc5.18":"Room5.18",
"voc5.20":"Room5.20","voc5.21":"Room5.21", "voc5.22":"Room5.22", "voc5.23":"Room5.23","voc5.26":"Room5.26","voc5.34":"Room5.34",
"voc5.35":"Room5.35", "voc5.36":"Room5.36","voc5.44":"Room5.44","voc5.45":"Room5.45","voc5.48":"Room5.48","voc5.49":"Room5.49",
"voc5.50":"Room5.50", "voc5.51":"Room5.51","voc5.52":"Room5.52", "voc5.53":"Room5.53", "voc5.54":"Room5.54", "voc5.56":"Room5.56",
"voc5.57":"Room5.57", "voc5.58":"Room5.58", "voc5.59":"Room5.59","voc5.60":"Room5.60", "voc5.61":"Room5.61", "voc5.62":"Room5.62",
"voc5.63":"Room5.63", "voc5.64":"Room5.64", "voc5.65":"Room5.65","voc5.66":"Room5.66", "voc5.67":"Room5.67", "voc5.68":"Room5.68",
"voc5.69":"Room5.69", "voc5.70":"Room5.70", "voc5.71":"Room5.71"
}
co_and_rooms = {
"co5.01":"Room5.01", "co5.02":"Room5.02","co5.03":"Room5.03","co5.04":"Room5.04","co5.05":"Room5.05","co5.06":"Room5.06",
"co5.07":"Room5.07", "co5.08":"Room5.08","co5.09":"Room5.09", "co5.10":"Room5.10", "co5.11":"Room5.11", "co5.12":"Room5.12",
"co5.13":"Room5.13","co5.14":"Room5.14", "co5.15":"Room5.15","co5.16":"Room5.16","co5.17":"Room5.17","co5.18":"Room5.18",
"co5.20":"Room5.20","co5.21":"Room5.21", "co5.22":"Room5.22", "co5.23":"Room5.23","co5.26":"Room5.26","co5.34":"Room5.34",
"co5.35":"Room5.35", "co5.36":"Room5.36","co5.44":"Room5.44","co5.45":"Room5.45","co5.48":"Room5.48","co5.49":"Room5.49",
"co5.50":"Room5.50", "co5.51":"Room5.51","co5.52":"Room5.52", "co5.53":"Room5.53", "co5.54":"Room5.54", "co5.56":"Room5.56",
"co5.57":"Room5.57", "co5.58":"Room5.58", "co5.59":"Room5.59","co5.60":"Room5.60", "co5.61":"Room5.61", "co5.62":"Room5.62",
"co5.63":"Room5.63", "co5.64":"Room5.64", "co5.65":"Room5.65","co5.66":"Room5.66", "co5.67":"Room5.67", "co5.68":"Room5.68",
"co5.69":"Room5.69", "co5.70":"Room5.70", "co5.71":"Room5.71"
}


In [11]:
for sensor, room in temp_and_rooms.items():
    g.add((bldg[sensor], BRICK['isPointOf'], bldg[room]))
    g.add((bldg[sensor], BRICK['hasLocation'], bldg[room]))
    g.add((bldg[room], BRICK['hasPoint'], bldg[sensor]))
    g.add((bldg[sensor], BRICK['hasUnit'], BRICK["DEG_C"]))
for sensor, room in hum_and_rooms.items():
    g.add((bldg[sensor], BRICK['isPointOf'], bldg[room]))
    g.add((bldg[sensor], BRICK['hasLocation'], bldg[room]))
    g.add((bldg[room], BRICK['hasPoint'], bldg[sensor]))
for sensor, room in sound_and_rooms.items():
    g.add((bldg[sensor], BRICK['isPointOf'], bldg[room]))
    g.add((bldg[sensor], BRICK['hasLocation'], bldg[room]))
    g.add((bldg[room], BRICK['hasPoint'], bldg[sensor]))
for sensor, room in hcho_and_rooms.items():
    g.add((bldg[sensor], BRICK['isPointOf'], bldg[room]))
    g.add((bldg[sensor], BRICK['hasLocation'], bldg[room]))
    g.add((bldg[room], BRICK['hasPoint'], bldg[sensor]))
for sensor, room in airq_and_rooms.items():
    g.add((bldg[sensor], BRICK['isPointOf'], bldg[room]))
    g.add((bldg[sensor], BRICK['hasLocation'], bldg[room]))
    g.add((bldg[room], BRICK['hasPoint'], bldg[sensor]))
for sensor, room in light_and_rooms.items():
    g.add((bldg[sensor], BRICK['isPointOf'], bldg[room]))
    g.add((bldg[sensor], BRICK['hasLocation'], bldg[room]))
    g.add((bldg[room], BRICK['hasPoint'], bldg[sensor]))
for sensor, room in dust_and_rooms.items():
    g.add((bldg[sensor], BRICK['isPointOf'], bldg[room]))
    g.add((bldg[sensor], BRICK['hasLocation'], bldg[room]))
    g.add((bldg[room], BRICK['hasPoint'], bldg[sensor]))
for sensor, room in pir_and_rooms.items():
    g.add((bldg[sensor], BRICK['isPointOf'], bldg[room]))
    g.add((bldg[sensor], BRICK['hasLocation'], bldg[room]))
    g.add((bldg[room], BRICK['hasPoint'], bldg[sensor]))
for sensor, room in mqtwo_and_rooms.items():
    g.add((bldg[sensor], BRICK['isPointOf'], bldg[room]))
    g.add((bldg[sensor], BRICK['hasLocation'], bldg[room]))
    g.add((bldg[room], BRICK['hasPoint'], bldg[sensor]))
for sensor, room in mqthree_and_rooms.items():
    g.add((bldg[sensor], BRICK['isPointOf'], bldg[room]))
    g.add((bldg[sensor], BRICK['hasLocation'], bldg[room]))
    g.add((bldg[room], BRICK['hasPoint'], bldg[sensor]))
for sensor, room in mqfive_and_rooms.items():
    g.add((bldg[sensor], BRICK['isPointOf'], bldg[room]))
    g.add((bldg[sensor], BRICK['hasLocation'], bldg[room]))
    g.add((bldg[room], BRICK['hasPoint'], bldg[sensor]))
for sensor, room in mqnine_and_rooms.items():
    g.add((bldg[sensor], BRICK['isPointOf'], bldg[room]))
    g.add((bldg[sensor], BRICK['hasLocation'], bldg[room]))
    g.add((bldg[room], BRICK['hasPoint'], bldg[sensor]))
for sensor, room in oxy_and_rooms.items():
    g.add((bldg[sensor], BRICK['isPointOf'], bldg[room]))
    g.add((bldg[sensor], BRICK['hasLocation'], bldg[room]))
    g.add((bldg[room], BRICK['hasPoint'], bldg[sensor]))
for sensor, room in cotwo_and_rooms.items():
    g.add((bldg[sensor], BRICK['isPointOf'], bldg[room]))
    g.add((bldg[sensor], BRICK['hasLocation'], bldg[room]))
    g.add((bldg[room], BRICK['hasPoint'], bldg[sensor]))
for sensor, room in notwo_and_rooms.items():
    g.add((bldg[sensor], BRICK['isPointOf'], bldg[room]))
    g.add((bldg[sensor], BRICK['hasLocation'], bldg[room]))
    g.add((bldg[room], BRICK['hasPoint'], bldg[sensor]))
for sensor, room in c2h5ch_and_rooms.items():
    g.add((bldg[sensor], BRICK['isPointOf'], bldg[room]))
    g.add((bldg[sensor], BRICK['hasLocation'], bldg[room]))
    g.add((bldg[room], BRICK['hasPoint'], bldg[sensor]))
for sensor, room in voc_and_rooms.items():
    g.add((bldg[sensor], BRICK['isPointOf'], bldg[room]))
    g.add((bldg[sensor], BRICK['hasLocation'], bldg[room]))
    g.add((bldg[room], BRICK['hasPoint'], bldg[sensor]))
for sensor, room in co_and_rooms.items():
    g.add((bldg[sensor], BRICK['isPointOf'], bldg[room]))
    g.add((bldg[sensor], BRICK['hasLocation'], bldg[room]))
    g.add((bldg[room], BRICK['hasPoint'], bldg[sensor]))

In [14]:
g.serialize("abacws-building-v2.ttl", format="ttl")

<Graph identifier=N2e07f2f9d7f4470bb4d420c0cda81f09 (<class 'brickschema.graph.Graph'>)>

In [13]:
with open("abacws-building-v2.ttl", "w") as f:
    # the Turtle format strikes a balance beteween being compact and easy to read
    f.write(g.serialize(format="ttl"))

In [15]:
print(f"Before: {len(g)} triples")



Before: 3673 triples


In [16]:
g.expand("owlrl")

<Graph identifier=N2e07f2f9d7f4470bb4d420c0cda81f09 (<class 'brickschema.graph.Graph'>)>

In [17]:
print(f"After: {len(g)} triples")

After: 3693 triples


In [19]:

g.expand(profile="shacl") # infers Brick classes from Brick tags

<Graph identifier=N2e07f2f9d7f4470bb4d420c0cda81f09 (<class 'brickschema.graph.Graph'>)>

In [20]:
print(f"After: {len(g)} triples")

After: 3693 triples


In [21]:
g.expand(profile="owlrl+shacl+vbis+shacl")

In [22]:
print(f"Inferred graph has {len(g)} triples")

Inferred graph has 4541 triples


In [24]:
valid, _, resultsText = g.validate()
if not valid:
    print("Graph is not valid!")
    print(resultsText)

In [28]:
g.serialize("abacws-building-v2.ttl", format="ttl")

<Graph identifier=N2e07f2f9d7f4470bb4d420c0cda81f09 (<class 'brickschema.graph.Graph'>)>

In [ ]:
# validating using externally-defined shapes
external = Graph()
external.load_file("other_shapes.ttl")
valid, _, report = g.validate(shape_graphs=[external])
print(f"Graph is valid? {valid}")
if not valid:
  print(report)

In [29]:

# perform SPARQL queries on the graph
res = g.query("""SELECT * WHERE {
    ?room rdf:type brick:Room .
}""")
for row in res:
    print(row)

(rdflib.term.URIRef('http://abacwsbuilding.cardiff.ac.uk/abacws#Room0.01'),)
(rdflib.term.URIRef('http://abacwsbuilding.cardiff.ac.uk/abacws#Room0.04'),)
(rdflib.term.URIRef('http://abacwsbuilding.cardiff.ac.uk/abacws#Room0.05'),)
(rdflib.term.URIRef('http://abacwsbuilding.cardiff.ac.uk/abacws#Room0.06'),)
(rdflib.term.URIRef('http://abacwsbuilding.cardiff.ac.uk/abacws#Room0.07'),)
(rdflib.term.URIRef('http://abacwsbuilding.cardiff.ac.uk/abacws#Room0.08'),)
(rdflib.term.URIRef('http://abacwsbuilding.cardiff.ac.uk/abacws#Room0.10'),)
(rdflib.term.URIRef('http://abacwsbuilding.cardiff.ac.uk/abacws#Room0.17'),)
(rdflib.term.URIRef('http://abacwsbuilding.cardiff.ac.uk/abacws#Room0.18'),)
(rdflib.term.URIRef('http://abacwsbuilding.cardiff.ac.uk/abacws#Room0.19'),)
(rdflib.term.URIRef('http://abacwsbuilding.cardiff.ac.uk/abacws#Room0.20'),)
(rdflib.term.URIRef('http://abacwsbuilding.cardiff.ac.uk/abacws#Room0.29'),)
(rdflib.term.URIRef('http://abacwsbuilding.cardiff.ac.uk/abacws#Room0.31'),)

In [27]:
# start a blocking web server with an interface for performing
# reasoning + querying functions
g.serve("localhost:8080")
# now visit in http://localhost:8080

 * Serving Flask app 'brickschema.web'
 * Debug mode: off


2023-09-27:16:43:05,035 INFO    [_internal.py:187] WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://localhost:8080
2023-09-27:16:43:05,039 INFO    [_internal.py:187] Press CTRL+C to quit


2023-09-27:16:44:01,218 INFO    [_internal.py:187] 127.0.0.1 - - [27/Sep/2023 16:44:01] "POST /query HTTP/1.1" 200 -


PREFIX bldg: <http://abacwsbuilding.cardiff.ac.uk/abacws#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX xml: <http://www.w3.org/XML/1998/namespace>
PREFIX wgs: <https://www.w3.org/2003/01/geo/wgs84_pos#>
PREFIX void: <http://rdfs.org/ns/void#>
PREFIX vcard: <http://www.w3.org/2006/vcard/ns#>
PREFIX vann: <http://purl.org/vocab/vann/>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX tag: <https://brickschema.org/schema/BrickTag#>
PREFIX ssn: <http://www.w3.org/ns/ssn/>
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX soda_hall: <https://brickschema.org/schema/1.0.2/building_example#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX sdo: <http://schema.org/>
PREFIX schema: <https://schema.org/>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
PREFIX qudtqk: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/sc

2023-09-27:16:44:13,255 INFO    [_internal.py:187] 127.0.0.1 - - [27/Sep/2023 16:44:13] "POST /query HTTP/1.1" 200 -


PREFIX bldg: <http://abacwsbuilding.cardiff.ac.uk/abacws#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX xml: <http://www.w3.org/XML/1998/namespace>
PREFIX wgs: <https://www.w3.org/2003/01/geo/wgs84_pos#>
PREFIX void: <http://rdfs.org/ns/void#>
PREFIX vcard: <http://www.w3.org/2006/vcard/ns#>
PREFIX vann: <http://purl.org/vocab/vann/>
PREFIX unit: <http://qudt.org/vocab/unit/>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX tag: <https://brickschema.org/schema/BrickTag#>
PREFIX ssn: <http://www.w3.org/ns/ssn/>
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX soda_hall: <https://brickschema.org/schema/1.0.2/building_example#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX sdo: <http://schema.org/>
PREFIX schema: <https://schema.org/>
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX ref: <https://brickschema.org/schema/Brick/ref#>
PREFIX qudtqk: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/sc

BRIKIFY

Arguments:

SOURCE: Path/URL to the source file [required]

Options:

--input-type TEXT: Supported input types: rac, table, rdf, haystack-v4

--brick PATH: Brick.ttl

--config PATH: Custom configuration file

--output PATH: Path to the output file

--serialization-format TEXT: Supported serialization formats: turtle, xml, n3, nt, pretty-xml, trix, trig and nquads [default: turtle]

--minify / --no-minify: Remove inferable triples [default: False]

--input-format TEXT: Supported input formats: xls, csv, tsv, url, turtle, xml, n3, nt, pretty-xml, trix, trig and nquads [default: turtle]

--building-prefix TEXT: Prefix for the building namespace [default: bldg]

--building-namespace TEXT: The building namespace [default: https://example.com/bldg#]

--site-prefix TEXT: Prefix for the site namespace [default: site]

--site-namespace TEXT: The site namespace [default: https://example.com/site#]

--install-completion: Install completion for the current shell.

--show-completion: Show completion for the current shell, to copy it or customize the installation.

--help: Show this message and exit.

In [ ]:
pip install brickschema[brickify]

In [ ]:
brickify --brick "brick\abacws-building-v2.ttl"

Curiously enough, we haven't actually made use of the Brick schema definition
yet. The Brick schema definition contains a set of rules and definitions. These
can help with:

- ensuring that classes and relationships are being used correctly
- allowing applications and users to query the Brick schema to better
  understand a class or relationship
- providing textual definitions of classes and relationships
- inferring classes from sets of tags (like Haystack) or from behavioral
  annotations (like "sensors that measure air temperature")

To understand what the Brick schema definition can give us, lets try finding
what kinds of air temperature sensors we have in our Brick model.

In [ ]:
sensors = g.query(
    """
    SELECT ?sensor WHERE {
    ?sensor rdf:type/rdfs:subClassOf* brick:Air_Temperature_Sensor
}
"""
)

This query uses the definition of the Brick class structure to find what
instances of Air_Temperature_Sensor exist as well as instances of any
*subclasses* of Air_Temperature_Sensor.  This kind of behavior is usually referred
to as "subtype polymorphism" and is analogous to what you'd find in Java.

*However*, our query returns no results because our Brick model doesn't know
what the Brick class hierarchy is yet.

In [ ]:
assert len(sensors) == 0

If we want to make use of the Brick schema definition, we need to "import" it.
You need a file called "Brick.ttl" on your computer (this can be obtained from
https://brickschema.org/resources/ or at https://github.com/BrickSchema/Brick/releases)

Assuming Brick.ttl is in the root directory of this repo, you can load it with the following.

In [ ]:
g.parse("../../Brick.ttl", format="ttl")

Now our query should execute and return one result (BLDG.VAV2-4.ZN-T)

In [ ]:

sensors = g.query(
    """SELECT ?sensor WHERE {
    ?sensor rdf:type brick:Temperature_Sensor
}"""
)
assert len(sensors) == 1

We can also expand Brick to add additional classes. Eventually, these suggestions should
be pushed upstream and contributed back to the community (see the Contribution guidelines at
https://github.com/BrickSchema/Brick/blob/master/CONTRIBUTING.md)

For now, lets define a new kind of zone called a Fire Zone. It is good practice when defining
new classes to "attach" them to existing classes in the Brick class structure through the
RDFS.subClassOf relationship. Here's the triples we need to define a new class: